
Código para o balanço térmico e análise de eficiencia de um painel solar térmico.

Parte I - Estimativa da radiação solar utilizando a biblioteca PVLIB

As informações da biblioteca PVLIB podem ser acessadas no seguinte endereço: https://pvlib-python.readthedocs.io/en/stable/index.html

A função utilizada para encontrar a irradiância total da biblioteca PVLIB pode ser acessada no seguinte endereço:
https://pvlib-python.readthedocs.io/en/stable/reference/generated/pvlib.irradiance.get_total_irradiance.html



In [2]:
!pip install pvlib
!pip install ht
!pip install CoolProp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 46.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.3/811.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 56.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 48.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import pvlib

# Criando uma série temporal com dados meteorológicos
tz = 'Etc/GMT+3'  # Fuso horário

#No Google maps, a latitude e a longitude podem ser obtidas clicando com o botão direito no local desejado. E utilizando o Google Earth, as informações de latitude e a longitude estão disponíveis no canto inferior direito da tela.
lat = -23.557787584590784  # Latitude. 
lon = -46.73445560946659 # Longitude
alt = 730  # Altitude em metros. Utilizando o Google Earth, no canto inferior direito pode-se obter a altitude local.

# Definindo as características da superfície inclinada
surface_tilt = -23  # Inclinação em graus
surface_azimuth = 180  # Azimute em graus

date_range = pd.date_range(start='2021-01-01 00:00:00', end='2022-12-31 23:59:59', freq='1min', tz=tz)
solpos = pvlib.solarposition.get_solarposition(date_range, lat, lon)
dni_extra = pvlib.irradiance.get_extra_radiation(date_range)
pressure = pvlib.atmosphere.alt2pres(alt)
airmass = pvlib.atmosphere.get_relative_airmass(solpos['apparent_zenith'])
pressure_corr = pvlib.atmosphere.get_absolute_airmass(airmass, pressure)
linke_turbidity = pvlib.clearsky.lookup_linke_turbidity(date_range, lat, lon)
solar_zenith = solpos['apparent_zenith']
solar_azimuth = solpos['azimuth']
diffuse_frac = pvlib.irradiance.get_ground_diffuse(surface_tilt, solar_azimuth)

# Calculando a irradiação solar em superfície inclinada com o modelo de Perez
dni = pvlib.irradiance.get_total_irradiance(surface_tilt, surface_azimuth, solar_zenith, solar_azimuth,
                                            dni_extra, linke_turbidity, diffuse_frac, pressure_corr,
                                            model='perez')

# Obtendo a irradiância total em superfície inclinada a partir do dicionário retornado
irrad_total = dni['poa_global']

# Calculando a média da irradiância total na superfície
irrad_total_media = dni['poa_global'].mean()

# Imprimindo o valor da média da irradiância total na superfície em W/m²
print(f"Irradiância total média na superfície: {irrad_total_media:.2f} W/m²")


Irradiância total média na superfície: 419.63 W/m²


Parte II - Thermal analysis of flat plate collectors.
As informações da biblioteca PVLIB podem ser acessadas no seguinte endereço: https://ht.readthedocs.io/en/latest/index.html

In [73]:
import ht
import CoolProp.CoolProp as CP

# Get fluid properties from CoolProp
fluid = input("Enter the fluid type (e.g. water, air): ")
T_inf = float(input("Enter the fluid temperature [°C]: "))
Ta = float(input("Enter the ambient temperature [°C]: "))
T_p = float(input("Enter the plate temperature [°C]: "))
p_inf = float(input("Enter the fluid pressure [bar]: "))
g = 9.81 #m/s^2
L = float(input("Enter the characteristic length of the plate [m]: "))
W = float(input("Enter the characteristic width of the plate [m]: "))

#Dependendo da correlação que será utilizada, as propriedades dos fluidos derão ser avaliadas na temperatura média do fuido, Tf =(T_inf+T_s)/2
T_f =(T_inf+T_p)/2
rho = CP.PropsSI("D", "T", T_f + 273.15, "P", p_inf * 1e5, fluid)
k = CP.PropsSI("conductivity", "T", T_f + 273.15, "P", p_inf * 1e5, fluid)
mu = CP.PropsSI("viscosity", "T", T_f + 273.15, "P", p_inf * 1e5, fluid)
nu = mu/rho
Pr = CP.PropsSI("Prandtl", "T", T_f + 273.15, "P", p_inf * 1e5, fluid)
cp = CP.PropsSI("CPMASS", "T", T_f + 273.15, "P", p_inf * 1e5, fluid)
beta = CP.PropsSI("ISOBARIC_EXPANSION_COEFFICIENT", "T", T_f + 273.15, "P", p_inf * 1e5, fluid)
alpha = k / (rho * cp)

A = L*W # area of the flat-plate solar collector
# Calculate Grashof number 
def calc_grashof(T_p, T_inf, g, L):
    Gr = (g * beta * (T_p - T_inf) * L**3) / (nu**2)
    return Gr

Gr = calc_grashof(T_p, T_inf, g, L)

# Calculate Rayleigh number 
def calc_rayleigh(T_p, T_inf, g, L, alpha):
    Ra = (g * beta * (T_p - T_inf) * L**3) / (nu*alpha)
    return Ra

Ra = calc_rayleigh(T_p, T_inf, g, L, alpha)

# Print fluid properties
print('The specific mass is: '+str(round(rho ,5))+' kg/m^3')
print('The thermal conditivity: '+str(round(k ,5))+' W/(m-K)')
print(f"The absolut viscosity: {mu} Pa.s")
print(f"The kinematic viscosity: {nu} m^2/s")
print('The Prandtl number is: '+str(round(Pr ,5)))
print('The specific heat is: '+str(round(cp ,5))+' J/kg-K')
print('The Vol. Expansivity is: '+str(round(beta ,7))+' 1/K')
print('The area of flat-plate solar collector is: '+str(round(A ,7))+' m^2')
print(f'The Grashof number is: {Gr}')
print(f'The Rayleigh number is: {Ra}')

Enter the fluid type (e.g. water, air): air
Enter the fluid temperature [°C]: 35
Enter the ambient temperature [°C]: 25
Enter the plate temperature [°C]: 40
Enter the fluid pressure [bar]: 1
Enter the characteristic length of the plate [m]: 1.2
Enter the characteristic width of the plate [m]: 0.8
The specific mass is: 1.12168 kg/m^3
The thermal conditivity: 0.02717 W/(m-K)
The absolut viscosity: 1.9046551029873684e-05 Pa.s
The kinematic viscosity: 1.6980394025470106e-05 m^2/s
The Prandtl number is: 0.70576
The specific heat is: 1006.7867 J/kg-K
The Vol. Expansivity is: 0.0032267 1/K
The area of flat-plate solar collector is: 0.96 m^2
The Grashof number is: 948505073.9964032
The Rayleigh number is: 669414812.2587363


#Function to calculate the  wind heat transfer coefficient (hw)

In [36]:

def wind_heat_transfer_coefficient(v, L_crv):
    """
    This function calculates the wind heat transfer coefficient according to McAdams' (1954) 
    recommendation by selecting the highest value between [5 and the correlation presented 
    by Mitchell (1976) for forced convection over buildings).
    
    Args:
    - v: wind velocity [m/s]
    - L_crv: L is the cube root of the house volume [m]
    
    Returns:
    - The greatest value found between the result of the function and the reference value.
    """
    
    hw = (8.6 * v ** 0.6) / L_crv ** 0.4
    return max(hw, 5)


#Function to calculate the top loss coefficient (Ut)

In [35]:
def top_loss_coefficient(N, epsilon_p, epsilon_c, Ta, Tpm, beta, hw):
    """
    Calculates the top loss coefficient (Ut) using the empirical equation developed by Klein (1979) following the basic procedure of Hottel and Woertz (1942) and Klein (1975).
    Inputs:
        N: number of glass covers
        epsilon_p: emittence plate
        epsilon_c: emittence of cover material
        T_a: ambient temperature [K]
        T_pm: mean plate temperature [K]
        hw: wind_heat_transfer_coefficient [W/(m^2.K)]
    Outputs:
        Ut: top loss coefficient [W/(m^2.K)]
"""

# Iteração para calcular U_t
    for i in range(max_iter):
    # Cálculo de f, C e e
        f = (1 + 0.089*h_w - 0.1166*h_w*epsilon_p)*(1 + 0.07866*N)
        beta_r = math.radians(beta)
        if beta < 70:
          C = 520*(1 - 0.000051*beta**2)
          e = 0.430*(1 - (100/T_pm))
        else:
          C = 520*(1 - 0.000051*70**2)
          e = 0.430*(1 - (100/T_pm))

        U_t = ((N/((C/T_pm)*((T_pm-T_a)/(N+f))**e))+(1/h_w))**-1 + (sigma*(T_pm**2+T_a**2)*(T_pm+T_a))/((1/(epsilon_p+0.00591*N*h_w))+((2*N+f-1+0.133*epsilon_p)/(epsilon_c))-N)
        return U_t


#Function to calculate the collector efficiency factor (F')

In [7]:
import math

def collector_efficiency_factor(UL, Wcb, D, Fdl, Ct, D_i, h_fi):
    # Calculate F'
    F_l = ((1/UL)/(Wcb*((1/(UL*(D+(Wcb-D)*Fdl))+(1/Ct)+(1/(math.pi*D_i*h_fi))))))
    F_0 = math.exp(-(UL*F_l*D)/Ct)
    F_prime = (1 - F_0) / (UL*F_l)
    return F_prime

# Calculating the thermal efficiency of the collector by obtaining Ut by the empirical method

# Input data

In [83]:
# Entrada de dados
sigma = 5.67e-8
k_ins = 0.045 # W/m.K
L_b = 0.025 # m
E_t = 0.025 # Edge insulation thickness
B_t = 0.0050 # Back insulation thickness
L_cb = 10 # m Collector bank length
W_cb = 3 # m Collector bank width
perimeter = 2*(L_cb+W_cb) # perimeter
A_c = L*W # área do coletor
A_cb = L_cb*W_cb # m^2 área da borda
m_dot = 0.02 # kg/s
cp = cp # J/kg.K Calor especifico 
D= 0.038 #where D is the outside tube diameter 
D_i = 0.03 #where Di is the inside tube diameter 
h_fi= 300 #[W/m^2K]. hfi is the heat transfer coefficient between the fluid and the tube wall.
C_b = 30

# Initial values
T_a = Ta+273.15 # em K. Temperatura ambiente°C
T_fi = T_inf+273.15 # K. fluid inlet temperature in the collector.

# A reasonable first guess for Tpm for liquid heating collectors operated at typical flow rates of 0.01 to 0.02 kg/m2 s is Tfi + 10∘C. For air heaters a reasonable first estimate is Tfi + 20∘C. Duffie e Backman(2020)
T_pm = (T_fi+10) # em K. Temperatura média da placa.

N = 2 # N é o número de coberturas.
epsilon_p = 0.87 # emissividade da placa
epsilon_c = 0.88 # emissividade da cobertura 1 (epsilon of glass = 0.88)
beta = 20 # degrees (inclinação do coletor).

# Initial values for iterative solution
v = 2 # wind speed in [m/s]
L_crv = 10 # L is the cube root of the house volume [m]
h_w= wind_heat_transfer_coefficient(v, L_crv) # wind heat transfer coefficient
print("O coeficiente de transferência de calor pelo vento de", round(h_w,2), "W/m^2.K")

O coeficiente de transferência de calor pelo vento de 5.19 W/m^2.K


In [100]:
import math

#loop iterativo 
max_iter = 1000  # Número máximo de iterações
tolerancia = 1e-6  # Tolerância para convergência

# Calculate initial U_t, U_b, U_e and U_L
U_t = top_loss_coefficient(N, epsilon_p, epsilon_c, T_a, T_pm, beta, h_w)
U_b = k_ins/L_b
U_e = ((k_ins/E_t)*perimeter*B_t)/(A_cb)
U_L = U_t + U_b + U_e

# Nova entrada de dados
FR = 0.8 #collector heat removal factor FR
F = 0.9 # Fin efficiency for tube-and-sheet solar collectors

# Cálculo inicial de Qu
alpha_abs = 0.89 #absortância
tau = 0.9 #transmitância
q_u = A_c*FR*((alpha_abs*tau)*irrad_total_media - (U_L)*(T_pm-T_a))

# Lista para armazenar valores de Ut e T_pm
Ut_values = []
T_pm_values = []

for i in range(max_iter):
    # Cálculo da nova temperatura média da placa
    T_pm_new = T_fi + ((q_u/A_c)/(FR*U_L))*(1-FR)
    
    # Calculate new U_L
    U_t = top_loss_coefficient(N, epsilon_p, epsilon_c, T_a, T_pm_new, beta, h_w)
    U_L_new = U_b + U_e + U_t
    
    # Calculate new FR and F_dl
    F_l = ((1/U_L_new)/(W_cb*((1/(U_L_new*(D+(W_cb-D)*F)+(1/C_b)+(1/(math.pi*D_i*h_fi)))))))
    FR_new = m_dot*cp/(A_c*U_L_new)*(1-math.exp(-(A_c*U_L_new*F_l)/(m_dot*cp)))
    F_dl_new = FR_new/F_l
    
    # Update T_pm and U_L for next iteration
    T_pm = T_pm_new
    U_L = U_L_new
    FR = FR_new
    F_dl = F_dl_new
    
    # Check convergence
    #if abs(T_pm - T_pm_new) < tolerancia:
        #break

# Verificar se os valores de F_R e F_dl convergiram
    if abs(FR_new - FR) < tolerancia and abs(F_dl_new - F_dl) < tolerancia:
        break

    if abs(T_pm_new - T_pm) < tolerancia:
        break

# Salva os valores em listas
#Ut_list.append(U_t)
#T_pm_list.append(T_pm)
effectiveness = q_u/(irrad_total_media*A_c)

print("O coeficiente de perda de calor pelo topo (Ut) pela eq. Klein (1979): ", round(U_t,2), "W/m^2.K")        
print(f"Final value of U_L is {U_L:.2f} W/m^2.K.")
print(f"Final value of FR is {FR:.2f}.")
print(f"Final value of F_dl is {F_dl:.2f}.")
print("A eficientia térmica do coletor é:", effectiveness)

O coeficiente de perda de calor pelo topo (Ut) pela eq. Klein (1979):  2.83 W/m^2.K
Final value of U_L is 4.64 W/m^2.K.
Final value of FR is 0.82.
Final value of F_dl is 0.91.
A eficientia térmica do coletor é: 0.45927291212424154


# Calculating the thermal efficiency of the collector by obtaining Ut by the resistance method

In [99]:
from ht.conv_free_enclosed import Rac_Nusselt_Rayleigh
Rac=Rac_Nusselt_Rayleigh(H=1.2, L=L, W=W, insulated=True)


FR = 0.8 #collector heat removal factor FR

Tceu = T_a
Tc1 = T_pm -2 # Considerando um delta de 2° entre a placa e a primeira cobertura.
Tc2 = (T_p) + 273.15
epsilon_c2 = epsilon_c
Le = 0.025#[m] Le é a distância entre a primeira cobertura e a placa absorvedora.
T_pc = (T_pm + T_a)/2

rho = CP.PropsSI("D", "T", T_pc, "P", p_inf * 1e5, fluid)
k = CP.PropsSI("conductivity", "T", T_pc, "P", p_inf * 1e5, fluid)
mu = CP.PropsSI("viscosity", "T", T_pc, "P", p_inf * 1e5, fluid)
nu = mu/rho # m^2/s
Pr = CP.PropsSI("Prandtl", "T", T_pc, "P", p_inf * 1e5, fluid)
Gr = calc_grashof(T_pm, Tc1, g, Le)
k_pc = CP.PropsSI("conductivity", "T", T_pc , "P", p_inf * 1e5, fluid)
Nu_H = ht.conv_free_enclosed.Nu_Nusselt_Rayleigh_Hollands(Pr, Gr, Rac=Rac) # O coeficiente de transferência de calor por convecção (hc_p_c1) pode ser encontrado a partir do número de Nusselt médio através da correlação desenvolvida por Hollands et al. (1976).

hc_p_c1 = ((Nu_H*k_pc)/Le)
hr_p_c1 = sigma*(T_pm+Tc1)*(T_pm**2+Tc1**2)/((1/epsilon_p)+(1/epsilon_c)-1) # coeficiente de transferência de calor por radiação da placa para a primeira cobertura.

R3= 1/(hc_p_c1+hr_p_c1)

hr_c2_a = sigma*epsilon_c2*(Tc2+Tceu)*(Tc2**2+Tceu**2) # coeficiente de transferência de calor por radiação da placa para a primeira cobertura.
R1 = 1/(h_w+hr_c2_a)


U_tr = 1/(R1+(2*R3))

U_b = k_ins/L_b
U_e = ((k_ins/E_t)*perimeter*B_t)/(A_cb)
U_Lr = U_tr + U_b + U_e

q_ur = A_c*FR*((alpha_abs*tau)*irrad_total_media - U_Lr*(T_pm-T_a))
effectiveness = q_ur/(irrad_total_media*A_c)

print("O número de Nusselt é:", round(Nu_H ,5))
print("O coeficiente de transferência de calor por convecção é:", round(hc_p_c1 ,3), "W/(m^2*K^4)")
print("O coeficiente de transferência de calor por radiação é:", round(hr_p_c1,3), "W/(m^2*K^4)")

print("O coeficiente de perda de calor pelo topo (U_tr) pelo método das resistências é:", round(U_tr,2), "W/m^2.K")        
print(f"Final value of U_Lr is {U_Lr:.2f} W/m^2.K.")
print("A eficientia térmica do coletor é:", effectiveness)



O número de Nusselt é: 16.03588
O coeficiente de transferência de calor por convecção é: 17.322 W/(m^2*K^4)
O coeficiente de transferência de calor por radiação é: 5.654 W/(m^2*K^4)
O coeficiente de perda de calor pelo topo (U_tr) pelo método das resistências é: 5.59 W/m^2.K
Final value of U_Lr is 7.40 W/m^2.K.
A eficientia térmica do coletor é: 0.3514421511698035
